# Label and describe using an LLM

Download and instantiate an LLM from Huggingface.

Load the LDA topic models. 

Prompt the LLM to generate a label and a description for each topic in the models.

In [1]:
import pandas as pd
import pickle
from transformers import pipeline
import nltk
nltk.download('punkt')


/home/atroncos/anaconda3/envs/arxiv_exp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /home/atroncos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Load the topic models fitted in a previous notebook.

* lda_gw: Gravitational Waves topics
* lda_cscl: Computation and Language topics

In [2]:
with open('../models/lda_gw.pickle', 'rb') as handle:
    lda_gw = pickle.load(handle)

Get a list of all topics in the model, each topic described by MAX_WORDS 

* The result is a list of topics. Each topic is represented by a tuple.
* The first element of the tuple is a topic number (int).
* The second element of the tuple is a list of tuples,
* Each tuple represents the words characterising he topic (string) and its corresponding probability (float)

In [3]:
MAX_WORDS = 30
# list[tuples<int, list[tuple<string, float>]>]
topics_gw = lda_gw.show_topics(num_words=MAX_WORDS, formatted=False)

## Setup a LLM pipeline for labelling

In [4]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

def predict_label(topic_str, min_length=1, max_length=12):
    #model_name = 'fabiochiu/t5-small-medium-title-generation'
    #model_name = 'deep-learning-analytics/automatic-title-generation'
    model_name = 'google-t5/t5-small'
    #model_name = 'sshleifer/distilbart-cnn-12-6'
    #model_name = "google/flan-t5-small"

    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

    input_text = [f"Summarize {topic_str}"]
    input_ids = tokenizer(input_text, max_length=512, truncation=True, return_tensors="pt").input_ids.to('cpu')

    outputs = model.generate(input_ids, num_beams=8, do_sample=True, min_length=min_length, max_length=max_length)
    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    predicted_label = nltk.sent_tokenize(decoded_outputs.strip())[0]

    return(predicted_label)  

### Using gated models

1. Go to huggingface, login, go to `settings/access tokens` 
2. Create a new READ token, save it to ../token.txt
3. Go here: https://huggingface.co/mistralai/Mistral-7B-v0.1 and accept the usage conditions

In [5]:
from huggingface_hub import login
with open('../token.txt', 'r') as handle:
    token = handle.read()
login(token=token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/atroncos/.cache/huggingface/token
Login successful


In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from accelerate import disk_offload

model_name = "mistralai/Mistral-7B-v0.1"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto").
disk_offload(model=model, offload_dir="offload")

def predict_label(topic_str, min_length=1, max_length=12):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    prompt = [f"Label: {topic_str}"]
    prompt = "My favourite condiment is"
    model_inputs = tokenizer([prompt], return_tensors="pt")
    model.to(device)
    generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
    decoded_outputs = tokenizer.batch_decode(generated_ids)[0]
    return(decoded_outputs)

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 34100.03it/s]


ValueError: You are trying to offload the whole model to the disk. Please use the `disk_offload` function instead.

In [7]:
def get_topic_str(topic):
    """Return the terms describing a topic as a string
    topic: list of tuples<string, float>
    """
    terms = [term[0] for term in topic[1]]
    resp = ', '.join([term[0] for term in topic[1]])
    return(resp)

In [8]:
def predict_topic_labels(topics):
    """
    Predict label for a list of topics.
    returns: dataFrame with columns: topic id, label
    """
    topics_range = [topic[0] for topic in topics]
    labels = []
    resp = []
    for topic_id in topics_range:
        print(f"Processing topic {topic_id} / {len(topics_range)}")
        topic_str = get_topic_str(topics[topic_id])
        label = predict_label(topic_str)
        labels.append(label)
        print(f"label: {label}")
    return(pd.DataFrame.from_dict({'topic': topics_range, 'label': labels}))

In [56]:
predict_topic_labels(topics_gw)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Processing topic 0 / 4
label: Summarize: detector, signal, data, noise
Processing topic 1 / 4


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


label: Summarize: binary, hole, mass, black
Processing topic 2 / 4


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


label: Summarize: model, spectrum, energy, dark
Processing topic 3 / 4
label: Summarize: field, theory, mode, gravity


,topic,label
0,0,"Summarize: detector, signal, data, noise"
1,1,"Summarize: binary, hole, mass, black"
2,2,"Summarize: model, spectrum, energy, dark"
3,3,"Summarize: field, theory, mode, gravity"


In [11]:
get_topic_str(topics_gw[0])

'detector, signal, data, noise, frequency, search, time, method, detection, pulsar, sensitivity, based, source, ligo, interferometer, parameter, analysis, timing, model, space, new, test, present, sky, result, measurement, limit, laser, lisa, mode'

In [9]:
text = """
T5 is an encoder-decoder model and converts all NLP problems into a text-to-text format. It is trained using teacher forcing. This means that for training, we always need an input sequence and a corresponding target sequence. The input sequence is fed to the model using input_ids. The target sequence is shifted to the right, i.e., prepended by a start-sequence token and fed to the decoder using the decoder_input_ids. In teacher-forcing style, the target sequence is then appended by the EOS token and corresponds to the labels. The PAD token is hereby used as the start-sequence token. T5 can be trained / fine-tuned both in a supervised and unsupervised fashion.
"""
predict_label(text, 1, 100)

KeyboardInterrupt: 

In [17]:
predict_label(get_topic_str(topics_gw[0]))

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 58254.22it/s]


ValueError: You are trying to offload the whole model to the disk. Please use the `disk_offload` function instead.

In [60]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")
model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small")

task_prefix = "translate English to German: "
sentences = ["The house is wonderful.", "I like to work in NYC."]

inputs = tokenizer([task_prefix + sentence for sentence in sentences], return_tensors="pt", padding=True)
output_sequences = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    do_sample=False,
)

print(tokenizer.batch_decode(output_sequences, skip_special_tokens=True))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


['Das Haus ist wunderbar.', 'Ich arbeite gerne in NYC.']


/home/atroncos/anaconda3/envs/arxiv_exp/lib/python3.12/site-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [66]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")
model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small")

task_prefix = "summarize: "
sentences = ['detector signal data noise frequency search time method detection pulsar sensitivity based source ligo interferometer parameter analysis timing model space new test present sky result measurement limit laser lisa mode']

inputs = tokenizer([task_prefix + sentence for sentence in sentences], return_tensors="pt", padding=True)
output_sequences = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    do_sample=False,
)

print(tokenizer.batch_decode(output_sequences, skip_special_tokens=True))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


['detector signal data noise frequency search time method detection pulsar sensitivity based source']


In [73]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "t5-small"  # encoder/decoder model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

text = """
This is how an Apple Watch saved a man\'s life after detecting accidentIt all started when Gabe Burdett was waiting for his father Bob at their pre-designated location for some mountain biking at the Riverside State Park when he received a text alert from his dad\'s Apple Watch, saying it had detected a "hard fall".Burdett, from city of Spokane in Washington State later received another update from the Watch, saying his father had reached Sacred Heart Medical Center."We drove straight there but he was gone when we arrived. I get another update from the Watch saying his location has changed with a map location of SHMC. Dad flipped his bike at the bottom of Doomsday, hit his head and was knocked out until sometime during the ambulance ride," Burdett wrote in a Facebook post.The Watch notified 911 with the location and within 30 minutes, emergency medical services (EMS) took the injured Bob to the hospital."If you own an Apple Watch, set up your hard fall detection, it\'s not just for when you fall off a roof or a ladder," Burdett further posted."Had he fallen somewhere on the High Drive trails or another remote area, the location would have clued EMS in on where to find him. Amazing technology and so glad he had it," Burdett said.(function(d, s, id) { var js, fjs = d.getElementsByTagName(s)[0]; if (d.getElementById(id)) return; js = d.createElement(s); js.id = id; js.src = "//connect.facebook.net/en_US/sdk.js#xfbml=1&version=v2.6&appId=1530374180564359"; fjs.parentNode.insertBefore(js, fjs); }(document, "script", "facebook-jssdk")); There have been several examples where Apple Watch saved lives.A US doctor recently saved a person\'s life by using Apple Watch Series 4 on his wrist to detect atrial fibrillation (AFib) at a restaurant.An Apple Watch user in the UK was recently alerted about his low heart rate by the device. It revealed a serious heart condition that ultimately resulted in a surgery to fix the problem.(With inputs from IANS)
"""
text = """
detector signal data noise frequency search time method detection pulsar sensitivity based source ligo interferometer parameter analysis timing model space new test present sky result measurement limit laser lisa mode
"""

input_ids = tokenizer.encode(
    'summarize: ' + text, # specify task by adding a prefix
    return_tensors='pt', max_length=512, truncation=True
)

summary_ids = model.generate(input_ids, max_length=25)
summary = tokenizer.decode(
    summary_ids[0], skip_special_tokens=True)

print(summary)

detector signal data noise frequency search time method detection pulsar sensitivity based source ligo interferometer


In [120]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the GPT-2 model and tokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# List of terms describing the topic
#terms = 'detector, signal, data, noise, frequency, search, time, method, detection, pulsar, sensitivity, based, source, ligo, interferometer, parameter, analysis, timing, model, space, new, test, present, sky, result, measurement, limit, laser, lisa, mode'
#terms = 'binary, hole, mass, black, star, merger, neutron, spin, model, source, rate, event, time, galaxy, compact, waveform, signal, stellar, system, parameter, sim, object, orbital, odot, simulation, massive, evolution, observation, ratio, high'
terms = 'model, spectrum, energy, dark, background, scale, matter, universe, primordial, ray, cosmic, inflation, neutrino, gamma, observation, transition, cmb, cosmological, phase, burst, high, emission, field, power, signal, early, density, parameter, large, time'

# Create the input text for the model
input_text = f"What is the name of the topic described by these terms: {terms}. Label:"

# Tokenize the input text
inputs = tokenizer.encode(input_text, return_tensors='pt', max_length=512, truncation=True)

# Generate text
outputs = model.generate(inputs, max_new_tokens=200, num_return_sequences=1, no_repeat_ngram_size=2, early_stopping=True)

# Decode the generated text
generated_label = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the generated label
print(generated_label)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is the name of the topic described by these terms: model, spectrum, energy, dark, background, scale, matter, universe, primordial, ray, cosmic, inflation, neutrino, gamma, observation, transition, cmb, cosmological, phase, burst, high, emission, field, power, signal, early, density, parameter, large, time. Label: "Model"

The term "model" is used to describe the model of a particle. It is a term that is often used in physics to refer to a set of parameters that are used by a system to determine the properties of particles. The term is also used for the particle's mass, mass-energy, and mass of its mass.
. A particle is defined as a mass that can be measured in a given amount of time, or as the mass density of an object. For example, a photon is an energy-density particle, which is measured by the energy of light. In the case of mass and energy density particles, the term mass is usually used. However, in the context of particle physics, it is sometimes used as an adjective to deno

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from accelerate import disk_offload
import torch

model_id="mistralai/Mistral-7B-Instruct-v0.2"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, low_cpu_mem_usage = True).cpu()
disk_offload(model=model, offload_dir="alpha")


Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████| 3/3 [00:05<00:00,  1.93s/it]


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm(

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [10]:
prompt = "My favourite condiment is"
model_inputs = tokenizer([prompt], return_tensors="pt").to("cpu")

In [11]:
generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
tokenizer.batch_decode(generated_ids)[0]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'<s> My favourite condiment is Sriracha. It adds flavour to almost any dish. I was searching for a simple recipe that requires no cooking and only three ingredients. I found this recipe, tried it, and it was love at first bite.\n\nI know it is only a few steps to put together but this sauce is a game changer in my kitchen. It is delicious on sandwiches, burgers, tacos, and even as a salad dressing. It takes only five minutes to'

In [15]:
%%time

terms = 'model, spectrum, energy, dark, background, scale, matter, universe, primordial, ray, cosmic, inflation, neutrino, gamma, observation, transition, cmb, cosmological, phase, burst, high, emission, field, power, signal, early, density, parameter, large, time'
prompt = f"What is the label of the topic in physics described by these terms: {terms}?"
model_inputs = tokenizer([prompt], return_tensors="pt").to("cpu")
generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
tokenizer.batch_decode(generated_ids)[0]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


CPU times: user 46min 18s, sys: 4min 22s, total: 50min 40s
Wall time: 16min 7s


'<s> What is the label of the topic in physics described by these terms: model, spectrum, energy, dark, background, scale, matter, universe, primordial, ray, cosmic, inflation, neutrino, gamma, observation, transition, cmb, cosmological, phase, burst, high, emission, field, power, signal, early, density, parameter, large, time? This is a physics and astronomy topic that may describe various aspects of the Big Bang theory, the structure and evolution of the universe, or the properties of subatomic particles and electromagnetic radiation. Some possible labels could be "Cosmology and Astrophysics," "Particle Physics," or "Electromagnetism and Cosmic Radiation," but this is a complex and multifaceted field with many interrelated topics, so a more precise label would depend on'